In [1]:
import lips
from lips import Particles, Particle
import numpy
from syngular import Field
from pyadic.finite_field import ModP, rationalise

from bgtrees.currents import J_μ, another_j
from bgtrees.finite_gpufields.finite_fields_tf import FiniteField
from bgtrees.finite_gpufields.operations import ff_dot_product
from bgtrees.metric_and_vertices import η
from bgtrees.settings import settings
from bgtrees.states import ε1, ε2, ε3, ε3c, ε4, ε4c, εxs, εxcs
from bgtrees.phase_space import random_phase_space_point, μ2

lips.spinor_convention = "asymmetric"
chosenP = 2**31 - 19
NTEST = 25
settings.run_tf_eagerly()

numpy.set_printoptions(threshold=numpy.inf, linewidth=numpy.inf)

2025-02-07 15:22:33.527128: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-07 15:22:33.556332: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-07 15:22:33.556359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-07 15:22:33.557266: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-07 15:22:33.562323: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def _generate_input(chosen_field, helconf, n=25):
    """Generate the momentum and polarization arrays using lips
    Returns the momentum and polarization as numpy array and the
    list of lips particle_lists
    """
    lmoms = []
    lpols = []
    lPs = []

    for seed in range(n):
        particle_list = Particles(len(helconf), field=chosen_field, seed=seed)
        particle_list = Particles([Particle(entry.four_mom, field=chosen_field) for entry in particle_list],
                                  field=chosen_field)
        particle_list.helconf = helconf

        # Prepare hte momentum array
        lm = []
        for oParticle in particle_list:
            lm.append(numpy.block([oParticle.four_mom, numpy.array([0, 0, 0, 0])]))
        lmoms.append(numpy.array(lm))

        # Prepare the polarization array
        lp = []
        momχ = particle_list.oRefVec.four_mom
        for index, helconf_index in enumerate(helconf):
            if helconf_index == "1" or helconf_index == "m":
                tmp = [ε1(lm[index], momχ, chosen_field)]
                # tmp = [εm(particle_list, index + 1), numpy.array([0, 0, 0, 0])]
            elif helconf_index == "2" or helconf_index == "p":
                tmp = [ε2(lm[index], momχ, chosen_field)]
                # tmp = [εp(particle_list, index + 1), numpy.array([0, 0, 0, 0])]
            elif helconf_index == "3":
                tmp = [ε3(lm[index], momχ, chosen_field)]
                # tmp = [numpy.array([0, 0, 0, 0, 0, 0, 1, 0])]
            elif helconf_index == "4":
                tmp = [ε4(lm[index])]
            else:
                raise Exception(f"Polarization request not understood for leg {index}: {helconf_index}.")
            lp.append(numpy.block(tmp))

        lpols.append(numpy.array(lp))
        lPs.append(particle_list)

    lmoms = numpy.array(lmoms)
    lpols = numpy.array(lpols)

    return lmoms, lpols, lPs

In [3]:
chosen_field = Field("finite field", chosenP, 1)
helconf = "ppmmm"
verbose = False

In [4]:
lmoms, lpols, lPs = _generate_input(chosen_field, helconf)

In [5]:
D = lmoms.shape[-1]
D

8

In [6]:
assert numpy.all(numpy.einsum("rim->rm", lmoms) == 0)
assert numpy.all(numpy.einsum("rim,mn,rin->ri", lmoms, η(D), lpols) == 0)

In [7]:
res_cpu = numpy.einsum("rm,rm->r", lpols[:, 0], J_μ(lmoms[:, 1:], lpols[:, 1:], put_propagator=False, verbose=verbose))
res_cpu

array([310719489 % 2147483629, 326776592 % 2147483629, 846490505 % 2147483629, 2038260253 % 2147483629, 1880010641 % 2147483629, 2026157625 % 2147483629, 2096906135 % 2147483629, 1388204694 % 2147483629, 1101718649 % 2147483629, 208358911 % 2147483629, 1730623862 % 2147483629, 476648099 % 2147483629, 419416627 % 2147483629, 2117053082 % 2147483629, 218913188 % 2147483629, 1250846245 % 2147483629, 929151615 % 2147483629, 1225836576 % 2147483629, 724363617 % 2147483629, 45084431 % 2147483629, 8777496 % 2147483629, 1312329228 % 2147483629, 485716476 % 2147483629, 1054361230 % 2147483629, 519506892 % 2147483629], dtype=object)

In [8]:
prev_setting = settings.use_gpu
settings.use_gpu = True

# Now make it into finite Finite Fields containers
def _make_to_container(array_of_arrays, p):
    """Make any array of arrays or list of list into a finite field container"""
    return FiniteField(array_of_arrays.astype(int), p)

ff_moms = _make_to_container(lmoms, chosenP)
ff_pols = _make_to_container(lpols, chosenP)

res_gpu = another_j(ff_moms[:, 1:], ff_pols[:, 1:], put_propagator=False, verbose=verbose)
res_gpu = ff_dot_product(ff_pols[:, 0], res_gpu)

settings.use_gpu = prev_setting
res_gpu

FiniteField(n=<tf.Tensor: shape=(25,), dtype=int64, numpy=array([ 310719489,  326776592,  846490505, 2038260253, 1880010641, 2026157625, 2096906135, 1388204694, 1101718649,  208358911, 1730623862,  476648099,  419416627, 2117053082,  218913188, 1250846245,  929151615, 1225836576,  724363617,   45084431,    8777496, 1312329228,  485716476, 1054361230,  519506892])>, p=2147483629)

In [9]:
res_cpu = numpy.einsum("rm,rm->r", lpols[:, 0], J_μ(lmoms[:, 1:], lpols[:, 1:], put_propagator=False, verbose=verbose))
res_cpu

array([310719489 % 2147483629, 326776592 % 2147483629, 846490505 % 2147483629, 2038260253 % 2147483629, 1880010641 % 2147483629, 2026157625 % 2147483629, 2096906135 % 2147483629, 1388204694 % 2147483629, 1101718649 % 2147483629, 208358911 % 2147483629, 1730623862 % 2147483629, 476648099 % 2147483629, 419416627 % 2147483629, 2117053082 % 2147483629, 218913188 % 2147483629, 1250846245 % 2147483629, 929151615 % 2147483629, 1225836576 % 2147483629, 724363617 % 2147483629, 45084431 % 2147483629, 8777496 % 2147483629, 1312329228 % 2147483629, 485716476 % 2147483629, 1054361230 % 2147483629, 519506892 % 2147483629], dtype=object)

In [10]:
res_analytic = numpy.array([oPs("(2[12]^4)/([12][23][34][45][51])") for oPs in lPs])
res_analytic

array([310719489 % 2147483629, 326776592 % 2147483629, 846490505 % 2147483629, 2038260253 % 2147483629, 1880010641 % 2147483629, 2026157625 % 2147483629, 2096906135 % 2147483629, 1388204694 % 2147483629, 1101718649 % 2147483629, 208358911 % 2147483629, 1730623862 % 2147483629, 476648099 % 2147483629, 419416627 % 2147483629, 2117053082 % 2147483629, 218913188 % 2147483629, 1250846245 % 2147483629, 929151615 % 2147483629, 1225836576 % 2147483629, 724363617 % 2147483629, 45084431 % 2147483629, 8777496 % 2147483629, 1312329228 % 2147483629, 485716476 % 2147483629, 1054361230 % 2147483629, 519506892 % 2147483629], dtype=object)

In [11]:
assert numpy.all(res_gpu.values.numpy().astype(int) == res_cpu.astype(int))
assert numpy.all(res_cpu.astype(int) == res_analytic.astype(int))

### D-Dim

In [12]:
from bgtrees.phase_space import random_phase_space_point
from bgtrees.states import all_states

In [13]:
m, D, field = 5, 6, Field("finite field", 2 ** 31 - 19, 1)
seeds = range(10)  # 10 replicas

In [14]:
lmoms = [random_phase_space_point(m, D, field, seed=seed) for seed in seeds]
lmomχ = [random_phase_space_point(2, 4, field, seed=seed)[0] for seed in seeds] # 4D ref. vectors

In [15]:
lstates = [[all_states(momD, momχ, field) for momD in momsD] for momsD, momχ in zip(lmoms, lmomχ)]

In [16]:
states_indices = [0, 0, 0, 3, 3]

In [17]:
lpols = [[_states[index] for (_states, _states_conj), index in zip(states, states_indices)] for states in lstates]

In [18]:
# just 1 replica
lmoms = numpy.array(lmoms)
lpols = numpy.array(lpols)

In [19]:
assert numpy.all(numpy.einsum("rim->rm", lmoms) == 0)
assert numpy.all(numpy.einsum("rim,mn,rin->ri", lmoms, η(D), lpols) == 0)

In [20]:
res_cpu = numpy.einsum("rm,rm->r", lpols[:, 0], J_μ(lmoms[:, 1:], lpols[:, 1:], put_propagator=False, verbose=verbose))
res_cpu

array([806027231 % 2147483629, 165957365 % 2147483629, 1676072925 % 2147483629, 497346580 % 2147483629, 398007974 % 2147483629, 2064460125 % 2147483629, 1271997044 % 2147483629, 405622683 % 2147483629, 2128457200 % 2147483629, 1294069580 % 2147483629], dtype=object)

In [21]:
prev_setting = settings.use_gpu
settings.use_gpu = True

# Now make it into finite Finite Fields containers
def _make_to_container(array_of_arrays, p):
    """Make any array of arrays or list of list into a finite field container"""
    return FiniteField(array_of_arrays.astype(int), p)

ff_moms = _make_to_container(lmoms, chosenP)
ff_pols = _make_to_container(lpols, chosenP)

res_gpu = another_j(ff_moms[:, 1:], ff_pols[:, 1:], put_propagator=False, verbose=verbose)
res_gpu = ff_dot_product(ff_pols[:, 0], res_gpu)

settings.use_gpu = prev_setting
res_gpu

FiniteField(n=<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 806027231,  165957365, 1676072925,  497346580,  398007974, 2064460125, 1271997044,  405622683, 2128457200, 1294069580])>, p=2147483629)

In [22]:
assert numpy.all(res_gpu.values.numpy().astype(int) == res_cpu.astype(int))